In [38]:
# Set up Colab environment
from google.colab import drive
drive.mount('/content/drive')

import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf /content/drive/MyDrive/spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# /content/spark-3.5.0-bin-hadoop3
# Start a SparkSession
import findspark
findspark.init()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [39]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Homesales").getOrCreate()

In [40]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(20)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [41]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')
df.createOrReplaceTempView('home_sales')


In [42]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
four_bedroom_avg_price = spark.sql("SELECT year(date) year, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 4 GROUP BY year ORDER BY year desc")
four_bedroom_avg_price.show()

+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2021|301819.44|
|2020|298353.78|
|2019| 300263.7|
+----+---------+



In [43]:
# prompt: What is the average price of a home for each year the home was built  that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

three_bedroom_three_bathroom_avg_price = spark.sql("SELECT year(date) year, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 and bathrooms = 3 GROUP BY year ORDER BY year desc")
three_bedroom_three_bathroom_avg_price.show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|292725.69|
|2021|294211.46|
|2020|294204.16|
|2019|287287.82|
+----+---------+



In [44]:
all_years = spark.sql("SELECT distinct year(date) year from home_sales order by year desc")
all_years.show()

+----+
|year|
+----+
|2022|
|2021|
|2020|
|2019|
+----+



In [45]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
three_bedroom_three_bathroom_two_floors_avg_price = spark.sql("SELECT year(date) year, ROUND(AVG(price), 2)\
    AS avg_price FROM home_sales WHERE bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000 GROUP BY year ORDER BY year desc")
three_bedroom_three_bathroom_two_floors_avg_price.show()

+----+---------+
|year|avg_price|
+----+---------+
|2022|290242.99|
|2021|296330.96|
|2020|292289.09|
|2019|289859.14|
+----+---------+



In [46]:
import time

# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
avg_price_view = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view desc")
avg_price_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.6602528095245361 seconds ---


In [19]:
view_count = spark.sql("select view, count(*) from home_sales group by 1 order by 1 desc")
view_count.show()

+----+--------+
|view|count(1)|
+----+--------+
| 100|      28|
|  99|      31|
|  98|      33|
|  97|      27|
|  96|      36|
|  95|      45|
|  94|      50|
|  93|      33|
|  92|      42|
|  91|      41|
|  90|      37|
|  89|      40|
|  88|      31|
|  87|      40|
|  86|      44|
|  85|      39|
|  84|      31|
|  83|      40|
|  82|      43|
|  81|      33|
+----+--------+
only showing top 20 rows



In [47]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [48]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [49]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
avg_price_view = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view desc")
avg_price_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.4511244297027588 seconds ---


In [50]:
# 10. Partition the home sales dataset by date_built field.
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partit_date_built")

In [51]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_partit_date_built")


In [52]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_home_sales')

In [53]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
avg_price_view = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM parquet_home_sales WHERE price >= 350000 GROUP BY view ORDER BY view desc")
avg_price_view.show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.6903328895568848 seconds ---


In [54]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")


DataFrame[]

In [55]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is still cached')
else:
  print('all clear')

all clear
